### GPIO

In [1]:
#インポート
import RPi.GPIO as GPIO
import dht11
import redis

class ght_inst:
    def __init__(self, pin):
        self.humid = -1
        self.temp = -1
        self.dht11 = dht11.DHT11(pin = 23)

    def measure(self):
        result = self.dht11.read()
        self.humid = result.humidity
        self.temp = result.temperature
    
    def call(self):
        self.measure()
        return {"t" : self.temp, "h" : self.humid}


In [2]:
# initialize GPIO
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.cleanup()

In [9]:
ght = ght_inst(23)
ght.call()

{'t': 29.7, 'h': 58.0}

### REDIS

In [18]:
import redis

In [19]:
r = redis.Redis(host="toukard.com", port=6379, db=0)

In [32]:
r.set("dht11" , "test")

True

### PUBSUB between python and python

In [16]:
r = redis.Redis(host="toukard.com", port=6379, db=0)
p = r.pubsub()

In [18]:
p.subscribe("dht11")

In [ ]:
for m in p.listen():
    print(m)

In [23]:
r.publish("dht11", s)

3

### transport

In [13]:
import json

In [10]:
di = ght.call()

In [14]:
s  = json.dumps(di)
s

'{"t": 0, "h": 0}'

### ELIXIR

In [ ]:
{:ok, conn} = Redix.start_link(host: "toukard,com", port: 6379) #これはできているので省略できる
{:ok, res} =  Redix.command(conn, ["get", "dht11"]) #これだけでよい

In [ ]:
{:ok, pubsub} = Redix.PubSub.start_link(host: "redis", port: 6379, database: 0)
{:ok, ref} = Redix.PubSub.subscribe(pubsub, "dht11", self())
receive do
  {:redix_pubsub, ^pubsub, ^ref, :message, %{channel: "dht11"} = properties} -> IO.inspect(properties.payload)
end

### FINAL

In [5]:
#インポート
import RPi.GPIO as GPIO
import dht11
import redis
import json

class ght_inst:
    def __init__(self, pin):
        self.humid = -1
        self.temp = -1
        self.inst = dht11.DHT11(pin = 23)

    def measure(self):
        result = self.inst.read()
        self.humid = result.humidity
        self.temp = result.temperature
    
    def call(self):
        self.measure()
        return json.dumps({"t" : self.temp, "h" : self.humid})

class redis_inst:
    def __init__(self, host, port=6379, db=0):
        self.inst = redis.Redis(host=host, port=port, db=db)
    def send(self, data, channel):
        #json(stringを受け取る)
        self.inst.publish(channel, data)

In [2]:
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.cleanup()

#インスタンス作成
ght_instance = ght_inst(23)
redis_instance = redis_inst("toukard.com")
channel = "dht11"

In [6]:
data = ght_instance.call()

#デバッグ
print(data)

#パブリッシュ
redis_instance.send(channel, data)


{"t": 29.8, "h": 58.0}


TypeError: send() takes 2 positional arguments but 3 were given

In [7]:
data

'{"t": 29.8, "h": 58.0}'

In [1]:
#! /usr/bin/python 


#インポート
import sys
#ユーザーのpipにパスを通す
sys.path.append("/home/toukaaoi/.local/lib/python3.9/site-packages")

import RPi.GPIO as GPIO
import redis
import time 
import json
import dht11


class ght_inst:
    def __init__(self, pin):
        self.humid = -1
        self.temp = -1
        self.inst = dht11.DHT11(pin = 23)

    def measure(self):
        result = self.inst.read()
        self.humid = result.humidity
        self.temp = result.temperature
    
    def call(self):
        self.measure()
        return json.dumps({"t" : self.temp, "h" : self.humid})

class redis_inst:
    def __init__(self, host, port=6379, db=0):
        self.inst = redis.Redis(host=host, port=port, db=db)
    def send(self, channel, data):
        #json(stringを受け取る)
        return self.inst.publish(channel, data)


def main():
    # initialize GPIO
    GPIO.setwarnings(False)
    GPIO.setmode(GPIO.BCM)
    GPIO.cleanup()

    #インスタンス作成
    ght_instance = ght_inst(23)
    redis_instance = redis_inst("toukard.com")
    channel = "dht11"

    #デーモン用
    while (True):
        #データ取得
        data = ght_instance.call()
        #デバッグ
        #print(data)
        #パブリッシュ
        debug = redis_instance.send(channel, data)
        print(debug)
        #1秒待
        time.sleep(1)

if __name__ == "__main__":
    main()

3
3
3


KeyboardInterrupt: 

In [7]:
redis_instance.send()

AttributeError: 'redis_inst' object has no attribute 'publish'